In [1]:
import pandas as pd
import gspread
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
import pickle
import os
from datetime import datetime
import json
from gspread_dataframe import set_with_dataframe
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

In [2]:
#Connect to GGSheet
scope = ["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]
# Path to the token.pickle file where the user's access and refresh tokens are stored
TOKEN_PICKLE_FILE = 'token.pickle'
CREDENTIALS_FILE = r"C:\\Python work space\DA\\client_secret_576124395548-hmv76ft3pgqcn65mtoj5vr8um38soqsh.apps.googleusercontent.com.json"

# Authenticate using OAuth2
def authenticate():
    creds = None
    if os.path.exists(TOKEN_PICKLE_FILE):
        with open(TOKEN_PICKLE_FILE, 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_FILE, scope)
            creds = flow.run_local_server(port=0)
        with open(TOKEN_PICKLE_FILE, 'wb') as token:
            pickle.dump(creds, token)
    return creds

# Get authenticated client
creds = authenticate()
client = gspread.authorize(creds)

# Create a new Google Sheet or open an existing one
spreadsheet = client.open("Power Automate")
sheet = spreadsheet.worksheet("Sheet1")

In [3]:
demo_data = sheet.col_values(3)[1:]

In [4]:
demo_data

['1. Cá cược là hoạt động mà người chơi đặt tiền vào kết quả của sự kiện nào đó, như trận đấu thể thao hoặc trò chơi may rủi.\n2. Cá cược có thể được thực hiện thông qua các trang web cá cược trực tuyến hoặc tại các nhà cái truyền thống.\n3. Nhiều người tham gia cá cược để thêm phần hấp dẫn vào việc theo dõi trận đấu hoặc trò chơi và cũng để có cơ hội kiếm lời.\n4. Tuy nhiên, cá cược cũng mang theo rủi ro cao và người chơi cần phải có sự hiểu biết và kiểm soát tài chính của mình.\n5. Để tránh rủi ro và tiêu cực trong cá cược, người chơi cần biết cách đặt mức cược hợp lý và không nên đặt cược dựa trên may rủi.',
 '1. Bóng đá là một môn thể thao đầy sức hấp dẫn và gay cấn, thu hút hàng triệu người trên khắp thế giới.\n2. Các trận đấu bóng đá không chỉ góp phần tạo nên niềm vui, mà còn thể hiện sự gắn kết và đoàn kết giữa người hâm mộ.\n3. Các cầu thủ bóng đá luôn phải rèn luyện và tập luyện chăm chỉ để có thể cống hiến những trận đấu đỉnh cao.\n4. World Cup là giải đấu bóng đá lớn nhất v

In [5]:
driver = webdriver.Chrome()  # Assumes chromedriver is in PATH

results = []  # To store results from the webpage
driver.get('https://www.zerogpt.com')
try:
    # Loop through each entry in the Demo column
    for input_data in demo_data:  
        # Locate the textarea and input text
        textarea = driver.find_element(By.ID, 'textArea')
        textarea.clear()  # Clear the textarea if needed
        textarea.send_keys(input_data)  # Input your data
        time.sleep(5)

        # Locate and click the button
        button = driver.find_element(By.CLASS_NAME, 'scoreButton')
        button.click()

        # Wait for the result to load
        time.sleep(10)  # Adjust the sleep time as necessary

        # Get the page source after the JavaScript has executed
        html = driver.page_source

        # Use Beautiful Soup to parse the HTML
        soup = BeautifulSoup(html, 'html.parser')

        # Find all matching <span> elements
        spans = soup.find_all('span', class_='header-text text-center')

        # Check if there are at least two spans found
        if len(spans) > 1:
            second_span_text = spans[1].get_text(strip=True)
            results.append(second_span_text)  # Store the result
        else:
            results.append('Not Found')  # Store a default value if not found

finally:
    # Close the WebDriver
    driver.quit()


In [6]:
# Update the Google Sheet with results in column "AI %" (assuming it's column AI)
for i, result in enumerate(results):
    sheet.update_cell(i + 2, 4, result)  # Column 4 is AI (1-indexed)
print("Results uploaded successfully to column AI %.")

Results uploaded successfully to column AI %.
